# Trabalho TP4 Ex 2
## Grupo 27
### LCC 2024/2025
#### Gonçalo Gonçalves Barroso A102931
#### Rafaela Antunes Pereira A102527
#### Ricardo Eusebio Cerqueira A102878

In [1]:
%pip install z3-solver

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
from z3 import *
from pysmt.shortcuts import *
from pysmt.typing import *

### Problema 2
Este exercício é dirigido à prova de correção do algoritmo estendido de Euclides  apresentado no trabalho TP3


    1. Construa a asserção lógica que representa a pós-condição do algoritmo. Note que a definição da função gcd é   
$$
\gcd(a, b) \equiv \min \{\, r > 0 \,|\, \exists s, t \,\centerdot\, r = a \cdot s + b \cdot t \}
$$

    2. Usando a metodologia do comando havoc para o ciclo, escreva o programa na linguagem dos comandos anotados (LPA). Codifique a pós-condição do algoritmo com um comando assert .
    3. Construa codificações do programa LPA através de transformadores de predicados “strongest post-condition” e  prove a correção  do programa LPA.

    1. Construa a asserção lógica que representa a pós-condição do algoritmo. Note que a definição da função gcd é   
$$
\gcd(a, b) \equiv \min \{\, r > 0 \,|\, \exists s, t \,\centerdot\, r = a \cdot s + b \cdot t \}
$$

In [3]:
def prove(f):

    with Solver(name="z3") as s:
        s.add_assertion(Not(f))
        if s.solve():
            print("Failed to prove.")
        else:
            print("Proved.")

In [4]:
a = Symbol('a', INT)
b = Symbol('b', INT)
r = Symbol('r', INT)
rl = Symbol('rl', INT)
s = Symbol('s', INT)
sl = Symbol('sl', INT)
t = Symbol('t', INT)
tl = Symbol('tl', INT)
q = Symbol('q', INT)
x = Symbol('x', INT)
y = Symbol('y', INT)
z = Symbol('z', INT)

pre = And(And(GT(a, Int(0)), GT(b, Int(0))), Not(Equals(rl, Int(0))))
pos = And(
    Equals(r, Plus(Times(a,s), Times(b,t))),
    GT(r, Int(0)),
    ForAll([x,y,z], Implies(And(Equals(x, Plus(Times(a,y), Times(b,z))), GT(x, Int(0))), LE(r, x)))
)

invariante = And(
    Equals(r, Plus(Times(a, s), Times(b, t))),
    Equals(rl, Plus(Times(a, sl), Times(b, tl))),
    GT(rl, Int(0)),
)

prog = And(Equals(q, Div(r, rl)),
           Equals(r, rl),
           Equals(rl, Minus(r, Times(q, rl))),
           Equals(s, sl),
           Equals(sl, Minus(sl, Times(q, sl))),
           Equals(t, tl),
           Equals(tl, Minus(tl, Times(q, tl)))
)

spc = Implies(And(pre, prog), pos)
prove(spc)

Proved.


    2. Usando a metodologia do comando havoc para o ciclo, escreva o programa na linguagem dos comandos anotados (LPA). Codifique a pós-condição do algoritmo com um comando assert .


    3. Construa codificações do programa LPA através de transformadores de predicados “strongest post-condition” e  prove a correção  do programa LPA.


In [5]:
init = And(
    GT(a, Int(0)), GT(b, Int(0)),
    Equals(r, a),
    Equals(rl, b),
    Equals(s, Int(1)),
    Equals(sl, Int(0)),
    Equals(t, Int(0)),
    Equals(tl, Int(1))
)

sp = And(init, Exists([r, rl, s, sl, t, tl, q], invariante))

loop = And(
    Not(Equals(rl, Int(0))),
    Equals(q, Div(r, rl)),
    Equals(r, rl),
    Equals(rl, Minus(r, Times(q, rl))),
    Equals(s, sl),
    Equals(sl, Minus(s, Times(q, sl))),
    Equals(t, tl),
    Equals(tl, Minus(t, Times(q, tl)))
)

sp = Or(And(sp, loop), And(sp, Equals(rl, Int(0))))

prove(Implies(sp, pos))

Proved.
